In [6]:
import tensorflow as tf
import os

In [4]:
v1 = tf.get_variable("v1", shape=[3])
saver = tf.train.Saver()

W0921 15:23:54.631252 139980133480256 deprecation.py:506] From /home/abhishek/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
with tf.Session() as sess:
  # Restore variables from disk.
      saver.restore(sess, "/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer/replication1/model.ckpt")
      print("Model restored.")

ValueError: The passed save_path is not a valid checkpoint: /home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer/replication1/model-30.ckpt

In [8]:
print(os.getcwd())

/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings


In [21]:
class car():
    def __init__(self):
        print("Hello")
    def write(self):
        self.name = "abhi"
    def name(self):
        print("Name is ",self.name)

In [22]:
c1 = car()
c1.write()

Hello


In [24]:
c1.name

'abhi'